In [4]:
using Distributions
using Optim

In [5]:
function generate_data(N)
    delta1, delta2 = -3 , -2
    beta1, beta2 = [1 4], [2 3]
    rho = -2
    lambda = exp(rho)/(1+exp(rho))

    Y = zeros(N,2)
    X = zeros(N,2)

    for i=1:N
        x1,x2 = rand(Normal(0.0, 1.0),3)
        e1,e2 = rand(Logistic(),2)
        select = rand(Categorical([lambda, (1-lambda)]))

        #see logit response i.e. definitely 11
        eq = []
        for y_init=[[0 0],[1 1],[0 1],[1 0]]
            y1,y2 = sum(beta1.*[1 x1]) + delta1*y_init[2] + e1 >= 0, sum(beta2.*[1 x2]) +delta2*y_init[1] + e2 >= 0

            if false
                println( "$(1*y1) $(1*y2),$(y_init)" )
                println(sum(beta1.*[1 x1]) + delta1*y_init[2] + e1," ", sum(beta2.*[1 x2]) +delta2*y_init[1] + e2)
                println("-----")
            end

            if (y1==y_init[1]) & (y2==y_init[2])
                append!(eq,[y_init])
            end
        end
        
        y1, y2 = eq[1]
        if size(eq)[1] >= 2
            y1, y2 = eq[select]
        end
        Y[i,:] =[y1 y2]
        X[i,:] =[x1 x2]
    end
    return Y,X
end

Y,X = generate_data(200)

([1.0 1.0; 0.0 1.0; … ; 1.0 0.0; 0.0 1.0], [1.6487 0.341927; -1.41143 1.26136; … ; 1.82195 0.253276; -0.994247 -0.636967])

In [10]:
function ll_obj(params;xs=0,ys=0)
    n = size(xs)[1]
    k = Integer(size(X)[2]/2) + 1
    p = zeros(2,2,n)

    #solve for the conditional probability firm i does not enter
    #index 1- firm, index2 - firm -i's action
    for a_j=0:1
        util1 = params[1] .+ xs[:,1].*params[2]  .+ a_j.*params[3]
        util2 = params[4] .+ xs[:,2].*params[5]  .+ a_j.*params[6]
        p[1,a_j+1,:] = 1 ./(1 .+ exp.(util1))
        p[2,a_j+1,:] = 1 ./(1 .+ exp.(util2))
    end
    
    lambda = exp(params[7])/(1+exp(params[7]))
 
    #solve for probablity of nash
    mult_eq = max.(( p[1,2,:]  .- p[1,1,:] ) .* (  p[2,2,:] .- p[2,1,:] ),0)
    prob01 = ( p[1,2,:] ) .* ( 1 .- p[2,1,:] ) - (1-lambda).*mult_eq
    prob10 = ( 1 .- p[1,1,:] ) .* ( p[2,2,:] ) - (lambda).*mult_eq
    prob00 = p[1,1,:] .* p[2,1,:]
    prob11 = ( 1 .- p[1,2,:] ) .* ( 1 .- p[2,2,:] )
    
    #compute empirical likelihood
    p00 = (1 .- ys[:,1]).*(1 .- ys[:,2])
    p11 = ys[:,1].*ys[:,2]
    p10 = ys[:,1].* (1 .- ys[:,2])
    p01 = (1 .- ys[:,1]) .* ys[:,2]
    
    ll = p00 .* prob00 + p11 .*prob11 .+ p01 .* prob01 .+ p10 .* prob10

    return -sum(log.(max.(ll,0)))    
end

Y,X = generate_data(200)
ll_obj([1 4 -3 2 3 -2 -2];xs=X,ys=Y)

116.69809144615971

In [11]:
function maximum_likelihood(params;xs=0,ys=0)
    ll_trial(params) = ll_obj(params;xs=xs,ys=ys)
    return optimize(ll_trial, params, LBFGS())
end

result =  maximum_likelihood([1 4 -3 2 3 -2 -2.];xs=X,ys=Y)
print(result.minimizer)
print(result.minimum)

[0.574309 4.07428 -2.52794 3.02956 3.80365 -3.66736 -20.8372]113.30959263781392

In [14]:
function monte_carlo()
    trials = 400
    params_200 = zeros((7,trials))
    params_1000 = zeros((7,trials))
    for i =1:trials
        Y_200, X_200 = generate_data(200)
        trial_200 = maximum_likelihood([1 4 -3 2 3 -2 -1.];xs=X_200,ys=Y_200)
        params_200[:,i] = trial_200.minimizer

        
        Y_1000, X_1000 = generate_data(1000)
        trial_1000 = maximum_likelihood([1 4 -3 2 3 -2 -1.];xs=X_1000,ys=Y_1000)
        params_1000[:,i] = trial_1000.minimizer
    end
    return params_200,params_1000
end

results_200,results_1000 = monte_carlo()

([0.659844 1.4445 … 1.45934 0.838081; 4.18556 5.49594 … 4.3438 4.23483; … ; -1.65924 -2.56669 … -2.2556 -1.25848; -1.88496 -1.78197 … -24.8057 0.493511], [0.822538 0.842029 … 1.08534 1.01219; 4.10341 4.2823 … 4.33985 4.176; … ; -1.86776 -1.76599 … -2.34887 -1.83226; -23.1204 -5.04995 … -0.426949 -0.265269])

In [15]:
println(mean(results_200[7,:]))
lambda_200 = exp(mean(results_200[7,:]))/(1 + exp(mean(results_200[7,:])))
println(lambda_200)
var_200 = std(results_200[7,:]) *exp(mean(results_200[7,:]))/(1 + exp(mean(results_200[7,:])))^2

-11.035740878152005
1.6115049140606054e-5


0.00021832990137368114

In [16]:
println(mean(results_1000[7,:]))
lambda_1000 = exp(mean(results_1000[7,:]))/(1 + exp(mean(results_1000[7,:])))
println(lambda_1000)
var_200 = std(results_1000[7,:]) *exp(mean(results_1000[7,:]))/(1 + exp(mean(results_1000[7,:])))^2

-7.1736698439242765
0.0007659175317496951


0.007338383383743261

In [17]:
function print_results(params_200, params_1000)
       
    params_200[7,:] = exp.(params_200[7,:])./(1 .+ exp.(params_200[7,:]))
    params_1000[7,:] = exp.(params_1000[7,:])./(1 .+ exp.(params_1000[7,:]))
    
    println("\\begin{center}")
    println("\\begin{table}")
    println("\\normalsize")
    println("\\begin{tabular}{c|c|c|c|c|c}")
    print("& True & Mean & SD  &  Mean  & SD  \\\\\n")
    print("& &  (200 obs) &  (200 obs) &  (1000 obs) &  (1000 obs)  \\\\\n")
    println("\\hline \\hline")
    param_names = ["\$ \\alpha_1 \$","\$ \\beta_1 \$", "\$ \\Delta_1 \$", 
        "\$ \\alpha_2 \$", "\$ \\beta_2 \$", "\$ \\Delta_2 \$","\$ \\lambda \$" ]
    true_param = ["1", "4", "-3", "2", "3", "-2", ".25"]
    for i=1:7
        print("$(param_names[i]) &")
        print("$(true_param[i]) &")
        print(" $(round(mean(params_200[i,:]), digits=4)) &")
        print(" $(round(std(params_200[i,:]), digits=3)) &")
        print(" $(round(mean(params_1000[i,:]), digits=4)) &")
        print(" $(round(std(params_1000[i,:]), digits=3)) \\\\\n")
        if (i ==3) | (i==6)
            println( "\\hline \\hline")
        else
            println( "\\hline")
        end
        
    end
    println("\\end{tabular}")
    println("\\end{table}")
    println("\\end{center}")
end

print_results(results_200[:,:], results_1000[:,:])

\begin{center}
\begin{table}
\normalsize
\begin{tabular}{c|c|c|c|c|c}
& True & Mean & SD  &  Mean  & SD  \\
& &  (200 obs) &  (200 obs) &  (1000 obs) &  (1000 obs)  \\
\hline \hline
$ \alpha_1 $ &1 & 1.1419 & 0.498 & 1.0325 & 0.226 \\
\hline
$ \beta_1 $ &4 & 4.2314 & 0.724 & 4.0443 & 0.299 \\
\hline
$ \Delta_1 $ &-3 & -3.2037 & 0.821 & -3.0396 & 0.337 \\
\hline \hline
$ \alpha_2 $ &2 & 2.0427 & 0.487 & 1.9987 & 0.208 \\
\hline
$ \beta_2 $ &3 & 3.1573 & 0.526 & 3.025 & 0.217 \\
\hline
$ \Delta_2 $ &-2 & -2.1147 & 0.659 & -2.0241 & 0.266 \\
\hline \hline
$ \lambda $ &.25 & 0.2044 & 0.281 & 0.1577 & 0.153 \\
\hline
\end{tabular}
\end{table}
\end{center}
